<a href="https://colab.research.google.com/github/borundev/pytorch_examples/blob/master/Gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -Uqq git+https://github.com/borundev/pytorch_examples.git

In [ ]:
import pytorch_lightning as pl
from pl_bolts.datamodules.cifar10_datamodule import CIFAR10DataModule
from torch import nn
import torch
from pytorch_lightning.loggers import WandbLogger
import numpy as np
import torchvision.utils as vutils
import wandb
import torchvision
import psutil
import os
import torch.nn.functional as F
from sklearn.metrics import precision_recall_curve, roc_curve
import matplotlib.pyplot as plt

In [ ]:
from pytorch_examples.gan.models.generator.generator_dcgan import GeneratorDCGAN
from pytorch_examples.gan.models.discriminator.discriminator_dcgan import DiscriminatorDCGAN
from pytorch_examples.gan.models.gan import GAN

In [ ]:
Generator=GeneratorDCGAN
Discriminator=DiscriminatorDCGAN
DataModule=CIFAR10DataModule

In [ ]:
torch.cuda.is_available()

In [ ]:
dm = DataModule('.')
latent_dim=100
img_shape=dm.size()


In [ ]:
logger = WandbLogger(project='gan_cifar',name='colab-gpu')

generator=Generator(latent_dim=latent_dim, img_shape=img_shape)
discriminator=Discriminator(img_shape=img_shape)

model = GAN(*dm.size(),latent_dim=latent_dim, generator=generator, discriminator=discriminator)

In [ ]:

dm.prepare_data()
dm.setup()
dataloader =dm.train_dataloader()
real_batch = next(iter(dataloader))

gpus=1
device = 'cuda:0'


real_images=np.transpose(vutils.make_grid(real_batch[0][:6], padding=2, normalize=True).detach().numpy(),(1,2,0))
logger.experiment.log({'real_sample':[wandb.Image(real_images, caption='Real Images')]})

In [ ]:
trainer = pl.Trainer(gpus=gpus,
                     max_epochs=50,
                     logger=logger,
                     progress_bar_refresh_rate=50
                     )

In [ ]:
trainer.fit(model, dm)

In [ ]:
wandb.finish()